![spotify_logo](images/spotify_logo3.png)

# Spotify API Scrape

## Helpful Links:
- [Spotify Web API - Authorization Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/)
- [Spotify API References](https://developer.spotify.com/documentation/web-api/reference/)

In [66]:
import config
import os
import requests
import json
from json import JSONEncoder
import pandas as pd
from datetime import datetime,timezone, timedelta
from tqdm import tqdm
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from urllib.parse import urlencode

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import music21
import xlsxwriter

## Step 1: Get Access Token

In [2]:
client_id = config.client_id
client_secret = config.client_secret

In [3]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

## Step 2: Pull Recently Played

In [4]:
base_url = 'https://api.spotify.com/v1/me/player/recently-played?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
personal_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"&limit=50", headers=headers)
personal_data.append(json.loads(r.text))

In [6]:
track_ids = []
album_names = []
artist_names = []
track_names = []

for i in range(len(personal_data[0]['items'])):
    track_ids.append(personal_data[0]['items'][i]['track']['id']) # Track ID
    album_names.append(personal_data[0]['items'][i]['track']['album']['name']) # Album Name
    artist_names.append(personal_data[0]['items'][i]['track']['artists'][0]['name']) # Artist Name
    track_names.append(personal_data[0]['items'][i]['track']['name']) # Track Name

In [7]:
list_dic={'track_id':track_ids,
          'track_name':track_names,
          'artist_name':artist_names,
          'album_name':album_names,
    }

In [69]:
df1=pd.DataFrame(list_dic)
df1.head(20)

,track_id,track_name,artist_name,album_name
0,6ta5yavnnEfCE4faU0jebM,Mr Blue,Catherine Feeny,Hurricane Glass
1,7dz48pntblPzJ9mTPiUH81,At The Door,The Strokes,At The Door
2,7Cy7wa746ywgkwIOFJtSor,Yoko Ono (Stripped with Matt Maeson),Moby Rich,Yoko Ono (Stripped)
3,4ZdmTNaBTErD8n9AQE0YaX,British Bombs,Declan McKenna,British Bombs
4,2nUV1fiD45RN6cQZ85GDc1,Salt (Nlmg),Ben Hon,The Light Left Over
5,3xMQOd1C3TXsjQ3pmzOmkC,Weight of Love,The Black Keys,Turn Blue
6,3Hx7RXqCS7Kzjy2ot2q1Gk,Fever,The Black Keys,Turn Blue
7,2MVwrvjmcdt4MsYYLCYMt8,Tighten Up,The Black Keys,Brothers
8,1PXsUXSM3LF2XNSkmIldPb,Little Black Submarines,The Black Keys,El Camino
9,25YlltWXRb9k7KbrEBRuhJ,Lo/Hi,The Black Keys,"""Let's Rock"""


## Step 3: Pull User's Top Artists & Tracks

In [9]:
username = config.username
client_id = config.client_id
client_secret = config.client_secret
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-top-read'

auth_token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [10]:
base_url = 'https://api.spotify.com/v1/me/top/tracks?'
#track_id = '6y0igZArWVi6Iz0rj35c1Y'

#2. Authentication
#3. Parameters -- would be stored with authentication
headers = {
    "Authorization": f"Bearer {auth_token}"
}

#4. Create an empty list
top_track_data = [] #would be good explore how to capture data at different points in time
r = requests.get(base_url+"time_range=medium_term"+"&&limit=50", headers=headers)
top_track_data.append(json.loads(r.text))

In [12]:
track_idss = []
album_namess = []
album_relase_datess = []
artist_namess = []
track_namess = []

for i in range(len(top_track_data[0]['items'])):
    track_idss.append(top_track_data[0]['items'][i]['id']) # Track ID
    album_namess.append(top_track_data[0]['items'][i]['album']['name']) # Album Name
    album_relase_datess.append(top_track_data[0]['items'][i]['album']['release_date'])
    artist_namess.append(top_track_data[0]['items'][i]['album']['artists'][0]['name']) # Artist Name
    track_namess.append(top_track_data[0]['items'][i]['name']) # Track Name

In [13]:
list_dic2={'track_id':track_idss,
           'track_name':track_namess,
           'album_name':album_namess,
           'artist_name':artist_namess,
           'album_relase_date':album_relase_datess,
    }

In [73]:
df2=pd.DataFrame(list_dic2)
df2.head()

,track_id,track_name,album_name,artist_name,album_relase_date
0,4pvb0WLRcMtbPGmtejJJ6y,exile (feat. Bon Iver),folklore,Taylor Swift,2020-07-24
1,6xZ4Q2k2ompmDppyeESIY8,Level of Concern,Level of Concern,Twenty One Pilots,2020-04-09
2,1ci0BoqpvH73L2TJzHhw9y,Modern Chemistry,Tell All Your Friend,Okey Dokey,2019-03-22
3,4w2tfK0JA8KrVegKnxukf4,The Kids Are Alright,404,Barns Courtney,2019-09-06
4,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,After Hours,The Weeknd,2020-03-20


## Step 4: Pull Track Info
- API Doc: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/

In [15]:
track_id_ls = df2['track_id'].tolist()

In [17]:
track_data = []
for id in tqdm(track_id_ls):
    base_url = f'https://api.spotify.com/v1/audio-features/{id}?'

    #2. Authentication
    #3. Parameters -- would be stored with authentication
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    r = requests.get(base_url, headers=headers)
    track_data.append(json.loads(r.text))

100%|██████████| 50/50 [00:08<00:00,  6.00it/s]


In [72]:
track_df = pd.json_normalize(track_data)
track_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.298,0.380,6,-8.426,1,0.0287,0.77800,0.000056,0.1100,0.152,75.602,audio_features,4pvb0WLRcMtbPGmtejJJ6y,spotify:track:4pvb0WLRcMtbPGmtejJJ6y,https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...,https://api.spotify.com/v1/audio-analysis/4pvb...,285634,4
1,0.754,0.583,4,-7.340,0,0.0432,0.32000,0.000150,0.1440,0.770,122.012,audio_features,6xZ4Q2k2ompmDppyeESIY8,spotify:track:6xZ4Q2k2ompmDppyeESIY8,https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...,https://api.spotify.com/v1/audio-analysis/6xZ4...,220051,4
2,0.548,0.608,5,-7.506,1,0.0265,0.00415,0.000025,0.0665,0.436,89.997,audio_features,1ci0BoqpvH73L2TJzHhw9y,spotify:track:1ci0BoqpvH73L2TJzHhw9y,https://api.spotify.com/v1/tracks/1ci0BoqpvH73...,https://api.spotify.com/v1/audio-analysis/1ci0...,230213,4
3,0.547,0.771,3,-4.722,1,0.0284,0.02660,0.000001,0.0819,0.674,122.036,audio_features,4w2tfK0JA8KrVegKnxukf4,spotify:track:4w2tfK0JA8KrVegKnxukf4,https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...,https://api.spotify.com/v1/audio-analysis/4w2t...,235773,4
4,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4


## Clean Up Track Data

## Update 'mode' to tell if track is major or minor

In [31]:
mod_dict = {0 : 'Minor',
            1: 'Major'}

In [36]:
track_df['mode'].replace(mod_dict, inplace=True)

## Update "key" to tell the actually key

In [38]:
music_dic = {
    0: 'C',
    1: 'C#/Db',
    2: 'D',
    3: 'D#/Eb',
    4: 'E',
    5: 'F',
    6: 'F#/Gb',
    7: 'G',
    8: 'G#/Ab',
    9: 'A',
    10: 'A#/Bb',
    11: 'B' 
}

In [39]:
track_df['key'].replace(music_dic, inplace=True)

## Add column duration in minutes/seconds

In [47]:
for i in range(len(track_df['id'])):
    millis=track_df['duration_ms'].iloc[i]
    millis = int(millis)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60
    minutes = int(minutes)

    track_df['duration_min'].iloc[i] = "%d:%d" % (minutes, seconds)

C:\Users\Nicholas\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Rename track id column

In [51]:
track_df=track_df.rename(columns = {'id':'track_id'})
track_df.head()                     

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_id,uri,track_href,analysis_url,duration_ms,time_signature,duration_min
0,0.298,0.380,F#/Gb,-8.426,Major,0.0287,0.77800,0.000056,0.1100,0.152,75.602,audio_features,4pvb0WLRcMtbPGmtejJJ6y,spotify:track:4pvb0WLRcMtbPGmtejJJ6y,https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...,https://api.spotify.com/v1/audio-analysis/4pvb...,285634,4,4:45
1,0.754,0.583,E,-7.340,Minor,0.0432,0.32000,0.000150,0.1440,0.770,122.012,audio_features,6xZ4Q2k2ompmDppyeESIY8,spotify:track:6xZ4Q2k2ompmDppyeESIY8,https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...,https://api.spotify.com/v1/audio-analysis/6xZ4...,220051,4,3:40
2,0.548,0.608,F,-7.506,Major,0.0265,0.00415,0.000025,0.0665,0.436,89.997,audio_features,1ci0BoqpvH73L2TJzHhw9y,spotify:track:1ci0BoqpvH73L2TJzHhw9y,https://api.spotify.com/v1/tracks/1ci0BoqpvH73...,https://api.spotify.com/v1/audio-analysis/1ci0...,230213,4,3:50
3,0.547,0.771,D#/Eb,-4.722,Major,0.0284,0.02660,0.000001,0.0819,0.674,122.036,audio_features,4w2tfK0JA8KrVegKnxukf4,spotify:track:4w2tfK0JA8KrVegKnxukf4,https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...,https://api.spotify.com/v1/audio-analysis/4w2t...,235773,4,3:55
4,0.514,0.730,C#/Db,-5.934,Major,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4,3:20


# Create Master Dataframe

In [53]:
master_df = pd.merge(df2, track_df,
                       how='left', on=['track_id'])

In [54]:
master_df

,track_id,track_name,album_name,artist_name,album_relase_date,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,duration_min
0,4pvb0WLRcMtbPGmtejJJ6y,exile (feat. Bon Iver),folklore,Taylor Swift,2020-07-24,0.298,0.3800,F#/Gb,-8.426,Major,...,0.1100,0.1520,75.602,audio_features,spotify:track:4pvb0WLRcMtbPGmtejJJ6y,https://api.spotify.com/v1/tracks/4pvb0WLRcMtb...,https://api.spotify.com/v1/audio-analysis/4pvb...,285634,4,4:45
1,6xZ4Q2k2ompmDppyeESIY8,Level of Concern,Level of Concern,Twenty One Pilots,2020-04-09,0.754,0.5830,E,-7.340,Minor,...,0.1440,0.7700,122.012,audio_features,spotify:track:6xZ4Q2k2ompmDppyeESIY8,https://api.spotify.com/v1/tracks/6xZ4Q2k2ompm...,https://api.spotify.com/v1/audio-analysis/6xZ4...,220051,4,3:40
2,1ci0BoqpvH73L2TJzHhw9y,Modern Chemistry,Tell All Your Friend,Okey Dokey,2019-03-22,0.548,0.6080,F,-7.506,Major,...,0.0665,0.4360,89.997,audio_features,spotify:track:1ci0BoqpvH73L2TJzHhw9y,https://api.spotify.com/v1/tracks/1ci0BoqpvH73...,https://api.spotify.com/v1/audio-analysis/1ci0...,230213,4,3:50
3,4w2tfK0JA8KrVegKnxukf4,The Kids Are Alright,404,Barns Courtney,2019-09-06,0.547,0.7710,D#/Eb,-4.722,Major,...,0.0819,0.6740,122.036,audio_features,spotify:track:4w2tfK0JA8KrVegKnxukf4,https://api.spotify.com/v1/tracks/4w2tfK0JA8Kr...,https://api.spotify.com/v1/audio-analysis/4w2t...,235773,4,3:55
4,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,After Hours,The Weeknd,2020-03-20,0.514,0.7300,C#/Db,-5.934,Major,...,0.0897,0.3340,171.005,audio_features,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4,3:20
5,6xQpOC55lufcqXvSzp7GTb,Hard To Be Alone,Hard To Be Alone,Barns Courtney,2020-07-10,0.414,0.4760,D#/Eb,-6.520,Major,...,0.0863,0.2400,94.087,audio_features,spotify:track:6xQpOC55lufcqXvSzp7GTb,https://api.spotify.com/v1/tracks/6xQpOC55lufc...,https://api.spotify.com/v1/audio-analysis/6xQp...,182053,4,3:2
6,1uddOsj7TyRA13hnS2yDyk,Can't Take My Eyes Off You,Can't Take My Eyes Off You / 3BadSoSad - Edit,Private Island,2019-11-15,0.614,0.6090,F,-5.837,Minor,...,0.1520,0.4290,123.996,audio_features,spotify:track:1uddOsj7TyRA13hnS2yDyk,https://api.spotify.com/v1/tracks/1uddOsj7TyRA...,https://api.spotify.com/v1/audio-analysis/1udd...,209000,4,3:29
7,2nUV1fiD45RN6cQZ85GDc1,Salt (Nlmg),The Light Left Over,Ben Hon,2019-04-05,0.570,0.5950,F#/Gb,-7.833,Major,...,0.1420,0.3360,136.588,audio_features,spotify:track:2nUV1fiD45RN6cQZ85GDc1,https://api.spotify.com/v1/tracks/2nUV1fiD45RN...,https://api.spotify.com/v1/audio-analysis/2nUV...,202840,4,3:22
8,3FRJFImdfX5NSY3QH3jI4u,Glistening,Something You Needed,Flipturn,2020-02-07,0.668,0.5740,D#/Eb,-12.066,Major,...,0.1150,0.2480,127.011,audio_features,spotify:track:3FRJFImdfX5NSY3QH3jI4u,https://api.spotify.com/v1/tracks/3FRJFImdfX5N...,https://api.spotify.com/v1/audio-analysis/3FRJ...,247559,4,4:7
9,3flgdcFBWI84DPi4s1jhhd,Savannah,Something You Needed,Flipturn,2020-02-07,0.535,0.0755,E,-20.142,Major,...,0.2380,0.4090,104.443,audio_features,spotify:track:3flgdcFBWI84DPi4s1jhhd,https://api.spotify.com/v1/tracks/3flgdcFBWI84...,https://api.spotify.com/v1/audio-analysis/3flg...,69072,4,1:9


# Create Excel File

In [68]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('PythonExport.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
master_df.to_excel(writer, index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']


for i, col in enumerate(master_df.columns):

    # find length of column i
    column_len = master_df[col].astype(str).str.len().max()
    # Setting the length if the column header is larger
    # than the max column value length
    column_len = max(column_len, len(col)) + 2
    
    # set the column length
    worksheet.set_column(i, i, column_len)

writer.save()